In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader

In [42]:
from pathlib import Path 
from typing import Tuple
from torchvision.io import read_image

root_folder = "./data/MelanomaDetectionLabeled/"

# setup transform with Imagenet normalization parameters
transform_test = transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transform_train = transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
])

In [43]:
# Test image load 
test_img_path = root_folder + "labeled/0_0.jpg"
im = read_image(test_img_path).type(torch.float32)/255.0
im.shape, im.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [44]:

class MelanomaDataset(Dataset):
    def __init__(self, folder, transform=None):
        super().__init__()
        self.folder = Path(folder).resolve()
        self.transform = transform
        self.img_path_list = list(self.folder.glob('*.jpg'))
        
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, index) -> Tuple:
        img_path = self.img_path_list[index]
        img = read_image(str(img_path)).type(torch.float32)/255.0        
        if self.transform:
            img = self.transform(img)
        label = img_path.stem.split('_')[1]
        return img, int(label) 
    
    
# test 

ds_train = MelanomaDataset(root_folder + "labeled", transform=transform_train)
ds_test = MelanomaDataset(root_folder + "test", transform=transform_test)

print("Train dataset size: ", len(ds_train))
print("Test dataset size: ", len(ds_test))

Train dataset size:  200
Test dataset size:  600


In [45]:
# Data Loader for Train & Test 
dl_train = DataLoader(ds_train, batch_size=64, shuffle=True)
dl_test = DataLoader(ds_train, batch_size=64, shuffle=False)

print("Train Loader Check:")
for X, y in dl_train:
    print(f"Shape of X [N, C, H, W]: {X.shape}, X.dtype: {X.dtype}")
    print(f"Shape of y: {y.shape}, y.dtype: {y.dtype}")
    break

print("\nTest Loader Check:")
for X, y in dl_test:
    print(f"Shape of X [N, C, H, W]: {X.shape}, X.dtype: {X.dtype}")
    print(f"Shape of y: {y.shape}, y.dtype: {y.dtype}")
    break

Train Loader Check:
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32]), X.dtype: torch.float32
Shape of y: torch.Size([64]), y.dtype: torch.int64

Test Loader Check:
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32]), X.dtype: torch.float32
Shape of y: torch.Size([64]), y.dtype: torch.int64
